In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from statsmodels.formula.api import logit
import statsmodels.formula.api as sm
import warnings
warnings.filterwarnings('ignore')

/Users/christams/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
Pro_WR_Path = 'Data/Data_CSVstats_Pro_Football_Ref_WR_Trail.csv'

data = pd.read_csv(Pro_WR_Path)

players = pd.read_csv('Data/Names.csv')

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
data = data.fillna(0)
data = data.drop(columns=["Unnamed: 0", "Pos_"])

In [6]:
def staus(data):
    different_staus_options = []
    temp = data.drop(columns=["Name_", "Unnamed: 2_level_0_Date", "Unnamed: 6_level_0_Tm", "Unnamed: 7_level_0_Unnamed: 7_level_1","Unnamed: 9_level_0_Result", "Unnamed: 8_level_0_Opp" ])
    for x in temp.columns:
        statuses = data[x].unique()
        for w in statuses:
            try:
                float(w)
            except:
                try:
                    float(w[:-1])
                except:
                    different_staus_options.append(w)
                    #print(w)
    return different_staus_options

In [7]:
data

,Def Interceptions_Int,Def Interceptions_PD,Def Interceptions_TD,Def Interceptions_Yds,Def. Snaps_Num,Def. Snaps_Pct,Fumbles_FF,Fumbles_FL,Fumbles_FR,Fumbles_Fmb,Fumbles_TD,Fumbles_Yds,Kick Returns_Rt,Kick Returns_TD,Kick Returns_Y/Rt,Kick Returns_Yds,Name_,Name__,Off. Snaps_Num,Off. Snaps_Pct,Passing_AY/A,Passing_Att,Passing_Cmp,Passing_Cmp%,Passing_Int,Passing_Rate,Passing_Sk,Passing_TD,Passing_Y/A,Passing_Yds,Passing_Yds.1,Punt Returns_Ret,Punt Returns_TD,Punt Returns_Y/R,Punt Returns_Yds,Punting_Blck,Punting_In20,Punting_In20%,Punting_NY/P,Punting_Net,Punting_Pnt,Punting_RetYds,Punting_TB,Punting_TB%,Punting_Y/P,Punting_Yds,Receiving_Ctch%,Receiving_Rec,Receiving_TD,Receiving_Tgt,Receiving_Y/R,Receiving_Y/Tgt,Receiving_Yds,Rushing_Att,Rushing_TD,Rushing_Y/A,Rushing_Yds,ST Snaps_Num,ST Snaps_Pct,Scoring_2PM,Scoring_FG%,Scoring_FGA,Scoring_FGM,Scoring_Pts,Scoring_Sfty,Scoring_TD,Scoring_XP%,Scoring_XPA,Scoring_XPM,Tackles_Ast,Tackles_Comb,Tackles_QBHits,Tackles_Solo,Tackles_TFL,Unnamed: 0_level_0_Rk,Unnamed: 10_level_0_GS,Unnamed: 11_level_0_Sk,Unnamed: 12_level_0_Sk,Unnamed: 13_level_0_Sk,Unnamed: 14_level_0_Status,Unnamed: 15_level_0_Sk,Unnamed: 16_level_0_Sk,Unnamed: 16_level_0_Status,Unnamed: 17_level_0_Sk,Unnamed: 17_level_0_Status,Unnamed: 18_level_0_Sk,Unnamed: 18_level_0_Status,Unnamed: 19_level_0_Sk,Unnamed: 1_level_0_Year,Unnamed: 20_level_0_Sk,Unnamed: 21_level_0_Sk,Unnamed: 21_level_0_Status,Unnamed: 22_level_0_Sk,Unnamed: 23_level_0_Sk,Unnamed: 23_level_0_Status,Unnamed: 24_level_0_Sk,Unnamed: 24_level_0_Status,Unnamed: 25_level_0_Sk,Unnamed: 25_level_0_Status,Unnamed: 26_level_0_Sk,Unnamed: 26_level_0_Status,Unnamed: 27_level_0_Status,Unnamed: 28_level_0_Sk,Unnamed: 28_level_0_Status,Unnamed: 29_level_0_Sk,Unnamed: 29_level_0_Status,Unnamed: 2_level_0_Date,Unnamed: 30_level_0_Sk,Unnamed: 30_level_0_Status,Unnamed: 31_level_0_Status,Unnamed: 32_level_0_Sk,Unnamed: 32_level_0_Status,Unnamed: 33_level_0_Sk,Unnamed: 33_level_0_Status,Unnamed: 34_level_0_Sk,Unnamed: 34_level_0_Status,Unnamed: 35_level_0_Sk,Unnamed: 35_level_0_Status,Unnamed: 36_level_0_Sk,Unnamed: 36_level_0_Status,Unnamed: 37_level_0_Sk,Unnamed: 37_level_0_Status,Unnamed: 38_level_0_Status,Unnamed: 39_level_0_Sk,Unnamed: 39_level_0_Status,Unnamed: 3_level_0_G#,Unnamed: 40_level_0_Sk,Unnamed: 40_level_0_Status,Unnamed: 41_level_0_Sk,Unnamed: 41_level_0_Status,Unnamed: 42_level_0_Status,Unnamed: 43_level_0_Sk,Unnamed: 43_level_0_Status,Unnamed: 44_level_0_Sk,Unnamed: 44_level_0_Status,Unnamed: 45_level_0_Status,Unnamed: 46_level_0_Status,Unnamed: 47_level_0_Status,Unnamed: 48_level_0_Status,Unnamed: 49_level_0_Status,Unnamed: 4_level_0_Week,Unnamed: 50_level_0_Status,Unnamed: 51_level_0_Status,Unnamed: 52_level_0_Status,Unnamed: 53_level_0_Status,Unnamed: 54_level_0_Status,Unnamed: 55_level_0_Status,Unnamed: 56_level_0_Status,Unnamed: 57_level_0_Status,Unnamed: 58_level_0_Status,Unnamed: 59_level_0_Status,Unnamed: 5_level_0_Age,Unnamed: 61_level_0_Status,Unnamed: 62_level_0_Status,Unnamed: 65_level_0_Status,Unnamed: 66_level_0_Status,Unnamed: 6_level_0_Tm,Unnamed: 7_level_0_Unnamed: 7_level_1,Unnamed: 8_level_0_Opp,Unnamed: 9_level_0_Result
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Decker, Eric",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0%,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010-09-12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0,23.181,0,0,0,0,DEN,@,JAX,L 17-24
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Decker, Eric",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0%,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010-09-19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,2.0,0,0,0,0,0,0,0,0,0,0,23.188,0,0,0,0,DEN,0,SEA,W 31-14
2,0,0,0,0,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,In

In [8]:
for index, name in enumerate(data["Name_"]):
    if name == 0:
        data["Name_"][index] = data["Name__"][index]
        data["Name__"][index] = 0
data = data.drop(columns = ["Name__"])

In [9]:
stats = staus(data)

In [10]:
set_res = set(stats) 
list_res = (list(set_res))
values_to_replace = []
 
for item in list_res: 
    print(item)
    values_to_replace.append(item)

Non-Football Injury
*
COVID-19 List
Injured Reserve
Did Not Play
Suspended
Exempt List
Physically Unable to Perform
Inactive


In [11]:
for x in values_to_replace:
    data[x] = 0

for w in data.columns:
    for index, z in enumerate(data[w]):
        if values_to_replace.count(z) > 0:
            #print(z, index)
            data[z][index] = 1

In [12]:
for x in values_to_replace:
    data = data.replace(x, 0)

In [13]:
for x in data.columns:
    length = len("Unnamed: ")
    if x[:length] == "Unnamed: ":
        real_col = x.split("_")
        data = data.rename(columns={ x : real_col[-1]})
        #print(real_col[-1])
        

In [14]:
#Delete Duplicate columns
data = data.loc[:,~data.columns.duplicated()].copy()

In [15]:
def result_maker(data):
    data["Win?"] = 0
    data["Lose?"] = 0
    data["Tie?"] = 0
    data["Score-not-counted"] = 0
    data["Tm-Score"] = 0
    data["Opp-Score"] = 0
    for index, x in enumerate(data["Result"]):
        if x[0] == "W" or x[0] == "L" or x[0] == "T":
            if x[0] == "W":
                data["Win?"][index] = 1
            elif x[0] == "L":
                data["Lose?"][index] = 1
            else:
                data["Tie?"][index] = 1
            ind_num = x[1:].split("-")
            if len(ind_num) == 2:
                data["Tm-Score"][index] = ind_num[0]
                data["Opp-Score"][index] = ind_num[1]
        else:
            data["Score-not-counted"][index] = 1

In [16]:
result_maker(data)
data = data.drop(columns=["Result"])

In [17]:
def seperate_date(data):
    data["Day"] = 0
    data["Month"] = 0
    data["No-Date"] = 0
    
    for index, x in enumerate(data["Date"]):
        Year_Month_Day = x.split("-")
        if len(Year_Month_Day) > 1:
            Month = Year_Month_Day[1]
            Day = Year_Month_Day[2]
        
            data["Day"][index] = Day
            data["Month"][index] = Month
        else:
            data["No-Date"][index] = 1
    

In [18]:
seperate_date(data)
data = data.drop(columns=["Date"])

In [19]:
def convert_percentage(data):
    for col in data.columns:
        for index, x in enumerate(data[col]):
            try:
                per = x[-1]
                if per == "%": 
                    data[col] = x[:-1]
            except:
                pass

In [20]:
convert_percentage(data)

In [21]:
def convert_to_number(data):
    temp = data.drop(columns = ["Name_", "Tm", "1", "Opp"])
    didnt_work = {}
    
    for col in temp.columns:
        #print(col)
        for x in data[col]:
            try:
                data[col] = x.astype(float)
            except:
                pass
            
        

In [22]:
convert_to_number(data)

In [23]:
def eliminate_Useless(data, num = 1, drop = False):
    limited = []
    for x in data.columns:
        amount = len(data[x].unique())
        if amount <= num:
            limited.append(x)
            print(x, amount, data[x].unique())
    if drop:
        data = data.drop(columns = limited)

In [24]:
eliminate_Useless(data, num = 5, drop = False)

Def. Snaps_Pct 1 ['0']
Off. Snaps_Pct 1 ['91']
Punting_Blck 4 [0 '0' 1.0 2.0]
Punting_In20 3 [0 '0' '1']
Punting_In20% 1 ['0.0']
Punting_NY/P 5 [0 '13.0' '24.0' '15.0' '17.3']
Punting_RetYds 2 [0 '0']
Punting_TB 3 [0 '0' '1']
Punting_TB% 1 ['0.0']
Receiving_Ctch% 1 ['0.0']
ST Snaps_Pct 1 ['0']
Scoring_FG% 4 [0 '100.0' 100.0 33.3]
Scoring_FGA 5 [0 '1' 1.0 2.0 6.0]
Scoring_FGM 4 [0 '1' 1.0 2.0]
Scoring_Sfty 3 [0 '1' '2']
Scoring_XP% 5 [0 '100.0' 100.0 50.0 92.3]
GS 1 [0]
Status 1 [0]
1 2 ['@' 0]
Non-Football Injury 2 [0 1]
* 2 [0 1]
COVID-19 List 2 [0 1]
Injured Reserve 2 [0 1]
Did Not Play 2 [0 1]
Suspended 2 [0 1]
Exempt List 2 [0 1]
Physically Unable to Perform 2 [0 1]
Inactive 2 [0 1]
Win? 2 [0 1]
Lose? 2 [1 0]
Tie? 2 [0 1]
Score-not-counted 2 [0 1]
No-Date 2 [0 1]


In [25]:
data = data.drop(columns=["Def. Snaps_Pct", 
                          "Off. Snaps_Pct", 
                          "Punting_RetYds", 
                          "Punting_In20%", 
                          "ST Snaps_Pct", 
                          "Punting_RetYds", 
                          "Punting_TB%", 
                          "Receiving_Ctch%", 
                          "GS", 
                          "ST Snaps_Pct", 
                          "Status"])

In [26]:
temp = data.drop(columns = {"Name_", "Tm", "1", "Opp"})
for x in temp.columns:
    if temp[x].dtype == object:
        try:
            data[x] = pd.to_numeric(data[x])
        except:
            print(x)

In [27]:
dk_points = pd.read_csv('Data/WeeklyPoints2014-2021.csv', delimiter = ";")
dk_points

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,DK_points,DK_salary
0,17,2014,1426,"Smith, Geno",QB,nyj,a,mia,30.12,5000
1,17,2014,1408,"Griffin III, Robert",QB,was,h,dal,25.34,5700
2,17,2014,1252,"Rodgers, Aaron",QB,gnb,h,det,24.44,8600
3,17,2014,1302,"Flacco, Joe",QB,bal,h,cle,23.58,6200
4,17,2014,1225,"Manning, Eli",QB,nyg,h,phi,23.16,7100
...,...,...,...,...,...,...,...,...,...,...
55003,1,2021,7002,Atlanta,Def,atl,h,phi,0,2000
55004,1,2021,7006,Chicago,Def,chi,a,lar,0,2800
55005,1,2021,7030,Tennessee,Def,ten,h,ari,0,2300
55006,1,2021,7014,Jacksonville,Def,jac,a,hou,-3,2700


In [28]:
title = ['Week', 'Year', 'GID', "DK_points", "DK_salary"]
index_to_remove = []
for name in title:
    for x in range(len(dk_points[name])):
        try:
            dk_points[name][x] = float(dk_points[name][x])
        except ValueError:
            dk_points[name][x] = 0  
            index_to_remove.append(x)
        

In [29]:
# Replace 'DK points' with NaN (or another value of your choice)
dk_points['Week'] = dk_points['Week'].replace('Week', float('nan'))
dk_points['Week'] = dk_points['Week'].astype(float)


dk_points['Year'] = dk_points['Year'].replace('Year', float('nan'))
dk_points['Year'] = dk_points['Year'].astype(float)

dk_points['DK_points'] = dk_points['DK_points'].replace('DK_points', float('nan'))
dk_points['DK_points'] = dk_points['DK_points'].astype(float)

dk_points['DK_salary'] = dk_points['DK_salary'].replace('DK_salary', float('nan'))
dk_points['DK_salary'] = dk_points['DK_salary'].astype(float)

In [30]:
def get_points(data):
    data["Dk_Points"] = 0
    data["Salary"] = 0
    
    for index, row in dk_points.iterrows():
        Name = row["Name"]
        Week = row["Week"]
        Year = row["Year"]
        
        Index = data[(data["Name_"] == Name) & (data["Year"] == float(Year)) & (data["Week"] == int(Week))].index
        
        data["Dk_Points"][Index] = row["DK_points"]
        data["Salary"][Index] = row["DK_salary"]
    return data

In [31]:
get_points(data)

,Def Interceptions_Int,Def Interceptions_PD,Def Interceptions_TD,Def Interceptions_Yds,Def. Snaps_Num,Fumbles_FF,Fumbles_FL,Fumbles_FR,Fumbles_Fmb,Fumbles_TD,Fumbles_Yds,Kick Returns_Rt,Kick Returns_TD,Kick Returns_Y/Rt,Kick Returns_Yds,Name_,Off. Snaps_Num,Passing_AY/A,Passing_Att,Passing_Cmp,Passing_Cmp%,Passing_Int,Passing_Rate,Passing_Sk,Passing_TD,Passing_Y/A,Passing_Yds,Passing_Yds.1,Punt Returns_Ret,Punt Returns_TD,Punt Returns_Y/R,Punt Returns_Yds,Punting_Blck,Punting_In20,Punting_NY/P,Punting_Net,Punting_Pnt,Punting_TB,Punting_Y/P,Punting_Yds,Receiving_Rec,Receiving_TD,Receiving_Tgt,Receiving_Y/R,Receiving_Y/Tgt,Receiving_Yds,Rushing_Att,Rushing_TD,Rushing_Y/A,Rushing_Yds,ST Snaps_Num,Scoring_2PM,Scoring_FG%,Scoring_FGA,Scoring_FGM,Scoring_Pts,Scoring_Sfty,Scoring_TD,Scoring_XP%,Scoring_XPA,Scoring_XPM,Tackles_Ast,Tackles_Comb,Tackles_QBHits,Tackles_Solo,Tackles_TFL,Rk,Sk,Year,G#,Week,Age,Tm,1,Opp,Non-Football Injury,*,COVID-19 List,Injured Reserve,Did Not Play,Suspended,Exempt List,Physically Unable to Perform,Inactive,Win?,Lose?,Tie?,Score-not-counted,Tm-Score,Opp-Score,Day,Month,No-Date,Dk_Points,Salary
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Decker, Eric",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2010.0,1.0,1.0,23.181,DEN,@,JAX,0,0,0,0,0,0,0,0,0,0,1,0,0,17,24,12,9,0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Decker, Eric",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2010.0,2.0,2.0,23.188,DEN,0,SEA,0,0,0,0,0,0,0,0,0,1,0,0,0,31,14,19,9,0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Decker, Eric",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2010.0,0.0,3.0,0.000,DEN,0,IND,0,0,0,0,0,0,0,0,1,0,1,0,0,13,27,26,9,0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Decker, Eric",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2010.0,4.0,4.0,23.202,DEN,@,TEN,0,0,0,0,0,0,0,0,0,1,0,0,0,26,20,3,10,0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Decker, Eric",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,2010.0,0.0,5.0,0.000,DEN,@,BAL,0,0,0,0,0,0,0,0,1,0,1,0,0,17,31,10,10,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52566,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Eskridge, Dee",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,2022.0,15.0,16.0,0.000,TEN,0,HOU,0,0,0,0,1,0,0,0,0,0,1,0,0,14,19,24,12,0,0.0,0.0
52567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Eskridge, Dee",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,2022.0,16.0,17.0,0.000

In [32]:
def filter_for_Prep(data):
    
    filtered_data = data[data["Year"] >= 2013]
    filtered_data = filtered_data.drop(columns = ["Rk", "G#"])
    filtered_data = filtered_data.loc[:,~data.T.duplicated(keep='first')]
    do_not_drop = ['Injured Reserve','Non-Football Injury','Suspended','Exempt List','Did Not Play','Physically Unable to Perform','Inactive','COVID-19 List','Tie?','Score-not-counted']
    eliminated = []
    eliminated_value = []
    for col in filtered_data.columns:
        num = (filtered_data[col] == 0).sum()/len(filtered_data)
        
        if num > 0.89 and do_not_drop.count(col) == 0:
            eliminated.append(col)
            eliminated_value.append(num)
            filtered_data = filtered_data.drop(columns=[col])
        
        
        
            
    return filtered_data, eliminated

In [33]:
filtered_data, eliminated = filter_for_Prep(data)

In [34]:
heights_and_weights = pd.read_csv('Data/data_WRs.csv', sep = ";")

heights_and_weights["Feet"] = 0
heights_and_weights["Inches"] = 0
heights_and_weights["Weight(lbs)"] = 0

index = 0
for x in heights_and_weights["Info"]:
    feet = x[0]
    inch = x[2]
    weight = x[5:8]
    
    heights_and_weights["Feet"][index] = feet
    heights_and_weights["Inches"][index] = inch
    heights_and_weights["Weight(lbs)"][index] = weight
    index += 1

In [35]:
index = 0
filtered_data = filtered_data.reset_index(drop=True)
filtered_data["Feet"] = 0
filtered_data["Inches"] = 0
filtered_data["Weight(lbs)"] = 0

dumb_players_with_no_weight = []
dumb_players_with_no_weight_index = []
for index, x in enumerate(filtered_data["Name_"]):
    try:
        indices = heights_and_weights[heights_and_weights['Name'] == x].index[0]

        filtered_data["Feet"][index] = heights_and_weights["Feet"][indices]
        filtered_data["Inches"][index] = heights_and_weights["Inches"][indices]
        filtered_data["Weight(lbs)"][index] = heights_and_weights["Weight(lbs)"][indices]
    except:
        dumb_players_with_no_weight.append(x)
        dumb_players_with_no_weight_index.append(index)
        
    index += 1

    
filtered_data.drop(labels = dumb_players_with_no_weight_index, axis = 0)
filtered_data = filtered_data.reset_index(drop=True)

In [36]:
Fan_Points_2013_2022 = pd.read_csv('Data/Fantasy_Points_2013-2022 - Sheet1.csv')
Fan_Points_2013_2022.TEAM = Fan_Points_2013_2022.TEAM.replace("LV", "LVR").replace("JAC", "JAX").replace("SF", "SFO").replace("TB", "TAM").replace("GB", "GNB").replace("KC", "KAN").replace("NO", "NOR").replace("NE", "NWE")

In [37]:
Fan_Points_2013_2022 = Fan_Points_2013_2022[Fan_Points_2013_2022["POS"] == "WR"].reset_index(drop=True)
Fan_Points_2013_2022["Convert_Name"] = 0


for index, name in enumerate(Fan_Points_2013_2022["PLAYER"]):
    name_seg = name.split(" ")
    if len(name_seg) == 2:
        Fan_Points_2013_2022["Convert_Name"][index] = name_seg[1] + ", " + name_seg[0]
    else:
        Fan_Points_2013_2022["Convert_Name"][index] = name_seg[1] + " " + name_seg[2] + ", " + name_seg[0]

In [38]:
def place_2022_scoring(filtered_data, Fan_Points_2013_2022):
    name_options = list(filtered_data["Name_"].unique())
    idoit = []
    accounted = 0
    amount = 0
    for index, row in Fan_Points_2013_2022.iterrows():
        name = row["Convert_Name"]
        week = 1
        if row["Year"] >= 2021:
            Games = row[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']]
        else:
            Games = row[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17']]
        if name in name_options:
            #print(name)
            for index, col in enumerate(Games.keys(), start=1):
                if col in Games:
                    try:
                        points = float(Games[col])
                    except:
                        points = 0
                        accounted += 1
                        
                    filtered_data.loc[(filtered_data["Year"] == int(row["Year"])) & (filtered_data["Name_"] == name) & (filtered_data["Week"] == week), 'Dk_Points'] = points
                amount += 1
                week += 1
        else:
            idoit.append(name)
    print(len(idoit)/len(list(filtered_data["Name_"].unique())) * 100, "% of the points sheet are not in", "\n", accounted/amount * 100, "% was accounted for")
    filtered_data["Dk_Points"] = filtered_data["Dk_Points"].astype(float)
    return idoit

In [39]:
idoit = place_2022_scoring(filtered_data, Fan_Points_2013_2022)
idoit

38.632162661737524 % of the points sheet are not in 
 34.20632385819227 % was accounted for


['Young, Dareke',
 'Gray, Danny',
 'Emezie, Emeka',
 'McCutcheon, Lance',
 'Gordon, Josh',
 'Thompson, Cody',
 'Wilson, Albert',
 'Morgan Jr., Stanley',
 'Ali, Josh',
 'Rogers, Chester',
 'Brown, Antonio',
 'Rogers, Chester',
 'Wesley, Antoine',
 'Wilson, Albert',
 'Grayson Jr., Cyril',
 'Grant Sr., Jakeem',
 'Ward, Greg',
 'Sanu Sr., Mohamed',
 'Johnson, Collin',
 'Miller, Anthony',
 'Gordon, Josh',
 'Williams, Preston',
 'Fuller V, William',
 'Snead IV, Willie',
 'Montgomery, D.J.',
 'Hill Jr., KJ',
 'Arcega-Whiteside, J.J.',
 'Morgan Jr., Stanley',
 'Saunders, C.J.',
 'Mickens, Jaydon',
 'Davis, Davion',
 'Hall, Marvin',
 'Black, Tarik',
 'Taylor, Malik',
 'Thompson, Cody',
 'Veasy, Jordan',
 'Baker, Kawaan',
 'Smith, Devin',
 'Mitchell, Osirus',
 'Akers, Landen',
 'Smith, Brandon',
 'Allison, Geronimo',
 'Hammond, Josh',
 'Hurns, Allen',
 'Adams, Rodney',
 'Stoner, Dillon',
 'Moore, David',
 'Cole, Matt',
 'Koski, J.J.',
 'Fuller V, William',
 'Ward, Greg',
 'Moore, David',
 'Brown

In [40]:
#filtered_data[filtered_data["Feet"] == 0]["Name_"].unique()

In [41]:
filtered_data.loc[(filtered_data["Year"] == 2022) & (filtered_data["Name_"] == "Williams, Mike") & (filtered_data["Week"] == 8)]

,Name_,Off. Snaps_Num,Receiving_Rec,Receiving_TD,Receiving_Tgt,Receiving_Y/R,Receiving_Y/Tgt,Receiving_Yds,ST Snaps_Num,Scoring_Pts,Scoring_TD,Year,Week,Age,Tm,1,Opp,Non-Football Injury,*,COVID-19 List,Injured Reserve,Did Not Play,Suspended,Exempt List,Physically Unable to Perform,Inactive,Win?,Lose?,Tie?,Score-not-counted,Tm-Score,Opp-Score,Day,Month,Dk_Points,Salary,Feet,Inches,Weight(lbs)


In [42]:
# forgotten_URLS = {'Brown, A.J.': 'https://www.pro-football-reference.com/players/B/BrowAJ00/gamelog/',
#                   'St. Brown, Amon-Ra': 'https://www.pro-football-reference.com/players/S/StxxAm00/gamelog/',
#                   'Smith, DeVonta': 'https://www.pro-football-reference.com/players/S/SmitDe07/gamelog/',
#                   'Pittman Jr., Michael': 'https://www.pro-football-reference.com/players/P/PittMi01/gamelog/',
#                   'Moore, DJ': 'https://www.pro-football-reference.com/players/M/MoorD.00/gamelog/',
#                   'Williams, Mike': 'https://www.pro-football-reference.com/players/W/WillMi07/gamelog/',
#                   'Davis, Gabe': 'https://www.pro-football-reference.com/players/D/DaviGa01/gamelog/',
#                   'Brown, Marquise': 'https://www.pro-football-reference.com/players/B/BrowMa04/gamelog/',
#                   'Osborn, K.J.': 'https://www.pro-football-reference.com/players/O/OsboKJ00/gamelog/',
#                   'Jones Jr., Marvin': 'https://www.pro-football-reference.com/players/J/JoneMa02/gamelog/',
#                   'Woods, Robert': 'https://www.pro-football-reference.com/players/W/WoodRo02/gamelog/',
#                   'Carter, DeAndre': 'https://www.pro-football-reference.com/players/C/CartDe02/gamelog/',
#                   'Claypool, Chase': 'https://www.pro-football-reference.com/players/C/ClayCh01/gamelog/',
#                   'Robinson, Demarcus': 'https://www.pro-football-reference.com/players/R/RobiDe01/gamelog/',
#                   'Parker, DeVante': 'https://www.pro-football-reference.com/players/P/ParkDe01/gamelog/',
#                   'Davis, Corey': 'https://www.pro-football-reference.com/players/D/DaviCo03/gamelog/',
#                   'Moore, Rondale': 'https://www.pro-football-reference.com/players/M/MoorRo02/gamelog/',
#                   'Robinson II, Allen': 'https://www.pro-football-reference.com/players/R/RobiAl02/gamelog/',
#                   'Jones, Julio': 'https://www.pro-football-reference.com/players/J/JoneJu02/gamelog/',
#                   'St. Brown, Equanimeous': 'https://www.pro-football-reference.com/players/S/St.BEq00/gamelog/',
#                   'Smith, Shi': 'https://www.pro-football-reference.com/players/S/SmitSh03/gamelog/',
#                   'Watson, Justin': 'https://www.pro-football-reference.com/players/W/WatsJu01/gamelog/',
#                   "Smith, Tre'Quan": 'https://www.pro-football-reference.com/players/S/SmitTr03/gamelog/',
#                   'Anderson, Chosen': 'https://www.pro-football-reference.com/players/A/AndeRo04/gamelog/',
#                   "Robinson, Wan'Dale": 'https://www.pro-football-reference.com/players/R/RobiWa01/gamelog/',
#                   'Thomas, Michael': 'https://www.pro-football-reference.com/players/T/ThomMi05/gamelog/',
#                   'Moore, Skyy': 'https://www.pro-football-reference.com/players/M/MoorSk01/gamelog/',
#                   'Miller, Scott': 'https://www.pro-football-reference.com/players/M/MillSc01/gamelog/',
#                   'Wilson Jr., Cedrick': 'https://www.pro-football-reference.com/players/W/WilsCe01/gamelog/',
#                   "Harry, N'Keal": 'https://www.pro-football-reference.com/players/H/HarrNK00/gamelog/',
#                   'Perriman, Breshad': 'https://www.pro-football-reference.com/players/P/PerrBr02/gamelog/',
#                   'Kennedy, Tom': 'https://www.pro-football-reference.com/players/K/KennTo01/gamelog/',
#                   'Smith, Jeff': 'https://www.pro-football-reference.com/players/S/SmitJe03/gamelog/',
#                   'Johnson, Marcus': 'https://www.pro-football-reference.com/players/J/JohnMa07/gamelog/',
#                   'Williams, Jameson': 'https://www.pro-football-reference.com/players/W/WillJa11/gamelog/',
#                   'Johnson, Brandon': 'https://www.pro-football-reference.com/players/J/JohnBr23/gamelog/',
#                   'Eskridge, Dee': 'https://www.pro-football-reference.com/players/P/PhilKy01/gamelog/',
#                   'Philips, Kyle': 'https://www.pro-football-reference.com/players/T/TaylTr02/gamelog/',
#                   'Taylor, Trent': 'https://www.pro-football-reference.com/players/B/BrowJo02/gamelog/',
#                   'Brown, John': 'https://www.pro-football-reference.com/players/T/ThomDe07/gamelog/',
#                   'Thompkins, Deven': 'https://www.pro-football-reference.com/players/S/StraMi03/gamelog/',
#                   'Strachan, Mike': 'https://www.pro-football-reference.com/players/T/ThomMi04/gamelog/',
#                   'Thomas, Mike': 'https://www.pro-football-reference.com/players/H/HartPe01/gamelog/',
#                   'Hart, Penny': 'https://www.pro-football-reference.com/players/E/EdwaBr01/gamelog/',
#                   'Edwards, Bryan': 'https://www.pro-football-reference.com/players/Y/YounDa02/gamelog/'}

In [43]:
# shit = {}
# for index, x in enumerate(forgotten_URLS):
#     shit[idoit[index]] = x
# shit

In [44]:
# forgotten = pd.DataFrame(data={"Name_":idoit, "First Name": first, "Last Name": last})
# #print(data["Name_"].unique(), "\n", data["Name_"])
# forgotten.to_csv('More_Forgotten_WR.csv', index=False, sep=";")


In [45]:
#last = []
#first = []

#for x in idoit:
#    sep = x.split(", ")
#    last.append(sep[1])
#    first.append(sep[0])

In [46]:
Offesive_Team_Stats = pd.read_csv('Data/NFL Team Season Stats - Offense Reg Season.csv', thousands=',')
Defensive_Team_Stats = pd.read_csv('Data/NFL Team Season Stats - Defense Reg Season.csv', thousands=',')
filtered_data["Tm"] = filtered_data["Tm"].replace("STL", "NOR").replace("OAK", "LVR").replace("SDG", "LAC")

In [47]:
def offensive_scoring_creation(filtered_data):
    filtered_data["Last_Off_Total_YDS"] = 0
    filtered_data["Last_Off_Total_YDS/G"] = 0
    filtered_data["Last_Off_Passing_YDS"] = 0
    filtered_data["Last_Off_Passing_YDS/G"] = 0
    filtered_data["Last_Off_Rushing_YDS"] = 0
    filtered_data["Last_Off_Rushing_YDS/G"] = 0
    filtered_data["Last_Off_Points_PTS"] = 0
    filtered_data["Last_Off_PointsPTS/G"] = 0
    for x in filtered_data.index:
        Team = filtered_data.loc[x]["Tm"]
        Year = filtered_data.loc[x]["Year"]
        row = Offesive_Team_Stats[(Offesive_Team_Stats["TEAM"] == Team) & (Offesive_Team_Stats["Year"] == Year)]

        # Ensure the conditions return only one row
        if len(row) == 1:
            filtered_data.loc[x, "Last_Off_Total_YDS"] = row["Total_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Off_Total_YDS/G"] = row["Total_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Off_Passing_YDS"] = row["Passing_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Off_Passing_YDS/G"] = row["Passing_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Off_Rushing_YDS"] = row["Rushing_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Off_Rushing_YDS/G"] = row["Rushing_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Off_Points_PTS"] = row["Points_PTS"].iloc[0]
            filtered_data.loc[x, "Last_Off_PointsPTS/G"] = row["PointsPTS/G"].iloc[0]

In [48]:
def def_scoring_creation(filtered_data):
    filtered_data["Last_Def_Total_YDS"] = 0
    filtered_data["Last_Def_Total_YDS/G"] = 0
    filtered_data["Last_Def_Passing_YDS"] = 0
    filtered_data["Last_Def_Passing_YDS/G"] = 0
    filtered_data["Last_Def_Rushing_YDS"] = 0
    filtered_data["Last_Def_Rushing_YDS/G"] = 0
    filtered_data["Last_Def_Points_PTS"] = 0
    filtered_data["Last_Def_PointsPTS/G"] = 0
    for x in filtered_data.index:
        Team = filtered_data.loc[x]["Tm"]
        Year = filtered_data.loc[x]["Year"]
        row = Defensive_Team_Stats[(Defensive_Team_Stats["TEAM"] == Team) & (Defensive_Team_Stats["Year"] == Year)]

        # Ensure the conditions return only one row
        if len(row) == 1:
            filtered_data.loc[x, "Last_Def_Total_YDS"] = row["Total_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Def_Total_YDS/G"] = row["Total_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Def_Passing_YDS"] = row["Passing_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Def_Passing_YDS/G"] = row["Passing_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Def_Rushing_YDS"] = row["Rushing_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Def_Rushing_YDS/G"] = row["Rushing_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Def_Points_PTS"] = row["Points_PTS"].iloc[0]
            filtered_data.loc[x, "Last_Def_PointsPTS/G"] = row["PointsPTS/G"].iloc[0]

In [49]:
#delete duplicate rows
filtered_data = filtered_data.drop_duplicates(keep='last')

In [50]:
def_scoring_creation(filtered_data)
offensive_scoring_creation(filtered_data)

In [51]:
def wr_week_one(data, team, opp, players, year, week):
    
    name_options = filtered_data.Name_.unique()
    temp_players = []
    
    for people in players:
        if people in name_options:
            temp_players.append(people)
    
    players = temp_players
    
    summary_team_stats = pd.DataFrame(data={"Last_Off_Total_YDS" : [0], 
                                            "Last_Off_Total_YDS/G" : [0], 
                                            "Last_Off_Passing_YDS" : [0],
                                            "Last_Off_Passing_YDS/G" : [0], 
                                            "Last_Off_Rushing_YDS" : [0], 
                                            "Last_Off_Rushing_YDS/G" : [0], 
                                            "Last_Off_Points_PTS" : [0], 
                                            "Last_Off_PointsPTS/G" : [0], 
                                            "Last_Def_Total_YDS" : [0],
                                            "Last_Def_Total_YDS/G" : [0], 
                                            "Last_Def_Passing_YDS" : [0], 
                                            "Last_Def_Passing_YDS/G" : [0], 
                                            "Last_Def_Rushing_YDS" : [0], 
                                            "Last_Def_Rushing_YDS/G" : [0], 
                                            "Last_Def_Points_PTS" : [0],
                                            "Last_Def_PointsPTS/G" : [0]})
    
    temp = pd.DataFrame(data = {"Name_":players})
    drop = ["Name_", "Tm", "1", "Opp", "Week", "Year", "Day", 
            'Tie?','Score-not-counted', "Weight(lbs)", "Inches", "Feet",
           "Last_Off_Total_YDS", "Last_Off_Total_YDS/G", "Last_Off_Passing_YDS",
           "Last_Off_Passing_YDS/G", "Last_Off_Rushing_YDS", "Last_Off_Rushing_YDS/G", 
           "Last_Off_Points_PTS", "Last_Off_PointsPTS/G", "Last_Def_Total_YDS",
           "Last_Def_Total_YDS/G", "Last_Def_Passing_YDS", "Last_Def_Passing_YDS/G", 
           "Last_Def_Rushing_YDS", "Last_Def_Rushing_YDS/G", "Last_Def_Points_PTS",
           "Last_Def_PointsPTS/G"]
    #temp = data.drop(columns = ["Name_", "Tm", "1", "Opp"])
    used_data = data.drop(columns = drop)
    
    for col in used_data.columns:
            if used_data[col].dtype != object:
                #temp[col + "_Count"] = 0
                temp[col + "_Average"] = 0.0
                temp[col + "_STD"] = 0.0
                temp[col + "_Min"] = 0.0
                temp[col + "_25%"] = 0.0
                temp[col + "_50%"] = 0.0
                temp[col + "_75%"] = 0.0
                temp[col + "_Max"] = 0.0
                
                
                temp["History_Opp_" + col + "_Count"] = 0.0
                temp["History_" + col + "_Average"] = 0.0
                temp["History_" + col + "_STD"] = 0.0
                temp["History_" + col + "_Min"] = 0.0
                temp["History_" + col + "_25%"] = 0.0
                temp["History_" + col + "_50%"] = 0.0
                temp["History_" + col + "_75%"] = 0.0
                temp["History_" + col + "_Max"] = 0.0
    
    
    
    seperate_df = list(summary_team_stats.columns)
    
    #temp["Feet"] = 0
    #temp["Inches"] = 0
    #temp["Weight(lbs)"] = 0
    temp["Actaul_Dk_Points"] = 0    
                
    #History of all games per player on team   
    index = 0
    for name in players:
        indiv_player = data[(data["Name_"] == name) & (data["Year"] < year)]
        indiv_player = indiv_player.drop(columns = drop)
       
        
        for col in indiv_player:
            if seperate_df.count(col) == 0:
                stats = indiv_player[col].describe()
                #temp[col + "_Count"][index] = stats[0]
                temp[col + "_Average"][index] = float(stats[1])
                temp[col + "_STD"][index] = float(stats[2])
                temp[col + "_Min"][index] = float(stats[3])
                temp[col + "_25%"][index] = float(stats[4])
                temp[col + "_50%"][index] = float(stats[5])
                temp[col + "_75%"][index] = float(stats[6])
                temp[col + "_Max"][index] = float(stats[7])   
        index += 1
    
    
    #History of all players on team vs oppenant
    index = 0 
    for name in players:
        last_games = data[(data["Name_"] == name) & (data["Opp"] == opp) & (data["Year"] < year)]
        last_games = last_games.drop(columns = drop)
        
        for col in last_games:
            if seperate_df.count(col) == 0:
                stats = last_games[col].describe()
                #temp["History_Opp_" + col + "_Count"][index] = stats[0]
                temp["History_" + col + "_Average"][index] = float(stats[1]) 
                temp["History_" + col + "_STD"][index] = float(stats[2]) 
                temp["History_" + col + "_Min"][index] = float(stats[3]) 
                temp["History_" + col + "_25%"][index] = float(stats[4]) 
                temp["History_" + col + "_50%"][index] = float(stats[5]) 
                temp["History_" + col + "_75%"][index] = float(stats[6])
                temp["History_" + col + "_Max"][index] = float(stats[7])   
        index += 1
    
    temp["Opp"] = opp
    temp["Team"] = team
    
    
    
    
    #Get the real points they made and bio
    index = 0
    for name in players:
        
        test_game = data[(data["Name_"] == name) & (data["Opp"] == opp) & (data["Year"] == year) & (data["Week"] == week)]

        # Assuming test_game contains only one row, otherwise this will error out.
        if len(test_game) == 1:
            #print(test_game["Dk_Points"].iloc[0])

            temp.loc[index, "Actaul_Dk_Points"] = test_game["Dk_Points"].iloc[0]
            temp.loc[index, "Month"] = test_game["Month"].iloc[0]
            temp.loc[index, "Year"] = test_game["Year"].iloc[0]
            temp.loc[index, "Week"] = test_game["Week"].iloc[0]
            #temp.loc[index, "Feet"] = test_game["Feet"].iloc[0]
            #temp.loc[index, "Inches"] = test_game["Inches"].iloc[0]
            #temp.loc[index, "Weight(lbs)"] = test_game["Weight(lbs)"].iloc[0]
        else:
            # Handle cases where test_game might have more than one row or no rows.
            print(f"Expected only one row for conditions, but found {len(test_game)} rows.")
        
        index += 1
    
    summary_team_stats["Last_Off_Total_YDS"] = float(test_game["Last_Off_Total_YDS"])
    summary_team_stats["Last_Off_Total_YDS/G"] = float(test_game["Last_Off_Total_YDS"])
    summary_team_stats["Last_Off_Passing_YDS"] = float(test_game["Last_Off_Passing_YDS"])
    summary_team_stats["Last_Off_Passing_YDS/G"] = float(test_game["Last_Off_Passing_YDS/G"])
    summary_team_stats["Last_Off_Rushing_YDS"] = float(test_game["Last_Off_Rushing_YDS"])
    summary_team_stats["Last_Off_Rushing_YDS/G"] = float(test_game["Last_Off_Rushing_YDS/G"])
    summary_team_stats["Last_Off_Points_PTS"] = float(test_game["Last_Off_Points_PTS"])
    summary_team_stats["Last_Off_PointsPTS/G"] = float(test_game["Last_Off_PointsPTS/G"])
    
    summary_team_stats["Last_Def_Total_YDS"] = float(test_game["Last_Def_Total_YDS"])
    summary_team_stats["Last_Def_Total_YDS/G"] = float(test_game["Last_Def_Total_YDS/G"])
    summary_team_stats["Last_Def_Passing_YDS"] = float(test_game["Last_Def_Passing_YDS"])
    summary_team_stats["Last_Def_Passing_YDS/G"] = float(test_game["Last_Def_Passing_YDS/G"])
    summary_team_stats["Last_Def_Rushing_YDS"] = float(test_game["Last_Def_Rushing_YDS"])
    summary_team_stats["Last_Def_Rushing_YDS/G"] = float(test_game["Last_Def_Rushing_YDS/G"])
    summary_team_stats["Last_Def_Points_PTS"] = float(test_game["Last_Def_Points_PTS"])
    summary_team_stats["Last_Def_PointsPTS/G"] = float(test_game["Last_Def_PointsPTS/G"])
        
        
        
        
        
    
    temp = temp.fillna(0)
    
    
    return temp, summary_team_stats

In [52]:
ready_data, team_data = wr_week_one(filtered_data, "LVR", "LAC", ["", "Adams, Davante"], year = 2022, week = 1)

In [53]:
ready_data

,Name_,Off. Snaps_Num_Average,Off. Snaps_Num_STD,Off. Snaps_Num_Min,Off. Snaps_Num_25%,Off. Snaps_Num_50%,Off. Snaps_Num_75%,Off. Snaps_Num_Max,History_Opp_Off. Snaps_Num_Count,History_Off. Snaps_Num_Average,History_Off. Snaps_Num_STD,History_Off. Snaps_Num_Min,History_Off. Snaps_Num_25%,History_Off. Snaps_Num_50%,History_Off. Snaps_Num_75%,History_Off. Snaps_Num_Max,Receiving_Rec_Average,Receiving_Rec_STD,Receiving_Rec_Min,Receiving_Rec_25%,Receiving_Rec_50%,Receiving_Rec_75%,Receiving_Rec_Max,History_Opp_Receiving_Rec_Count,History_Receiving_Rec_Average,History_Receiving_Rec_STD,History_Receiving_Rec_Min,History_Receiving_Rec_25%,History_Receiving_Rec_50%,History_Receiving_Rec_75%,History_Receiving_Rec_Max,Receiving_TD_Average,Receiving_TD_STD,Receiving_TD_Min,Receiving_TD_25%,Receiving_TD_50%,Receiving_TD_75%,Receiving_TD_Max,History_Opp_Receiving_TD_Count,History_Receiving_TD_Average,History_Receiving_TD_STD,History_Receiving_TD_Min,History_Receiving_TD_25%,History_Receiving_TD_50%,History_Receiving_TD_75%,History_Receiving_TD_Max,Receiving_Tgt_Average,Receiving_Tgt_STD,Receiving_Tgt_Min,Receiving_Tgt_25%,Receiving_Tgt_50%,Receiving_Tgt_75%,Receiving_Tgt_Max,History_Opp_Receiving_Tgt_Count,History_Receiving_Tgt_Average,History_Receiving_Tgt_STD,History_Receiving_Tgt_Min,History_Receiving_Tgt_25%,History_Receiving_Tgt_50%,History_Receiving_Tgt_75%,History_Receiving_Tgt_Max,Receiving_Y/R_Average,Receiving_Y/R_STD,Receiving_Y/R_Min,Receiving_Y/R_25%,Receiving_Y/R_50%,Receiving_Y/R_75%,Receiving_Y/R_Max,History_Opp_Receiving_Y/R_Count,History_Receiving_Y/R_Average,History_Receiving_Y/R_STD,History_Receiving_Y/R_Min,History_Receiving_Y/R_25%,History_Receiving_Y/R_50%,History_Receiving_Y/R_75%,History_Receiving_Y/R_Max,Receiving_Y/Tgt_Average,Receiving_Y/Tgt_STD,Receiving_Y/Tgt_Min,Receiving_Y/Tgt_25%,Receiving_Y/Tgt_50%,Receiving_Y/Tgt_75%,Receiving_Y/Tgt_Max,History_Opp_Receiving_Y/Tgt_Count,History_Receiving_Y/Tgt_Average,History_Receiving_Y/Tgt_STD,History_Receiving_Y/Tgt_Min,History_Receiving_Y/Tgt_25%,History_Receiving_Y/Tgt_50%,History_Receiving_Y/Tgt_75%,History_Receiving_Y/Tgt_Max,Receiving_Yds_Average,Receiving_Yds_STD,Receiving_Yds_Min,Receiving_Yds_25%,Receiving_Yds_50%,Receiving_Yds_75%,Receiving_Yds_Max,History_Opp_Receiving_Yds_Count,History_Receiving_Yds_Average,History_Receiving_Yds_STD,History_Receiving_Yds_Min,History_Receiving_Yds_25%,History_Receiving_Yds_50%,History_Receiving_Yds_75%,History_Receiving_Yds_Max,ST Snaps_Num_Average,ST Snaps_Num_STD,ST Snaps_Num_Min,ST Snaps_Num_25%,ST Snaps_Num_50%,ST Snaps_Num_75%,ST Snaps_Num_Max,History_Opp_ST Snaps_Num_Count,History_ST Snaps_Num_Average,History_ST Snaps_Num_STD,History_ST Snaps_Num_Min,History_ST Snaps_Num_25%,History_ST Snaps_Num_50%,History_ST Snaps_Num_75%,History_ST Snaps_Num_Max,Scoring_Pts_Average,Scoring_Pts_STD,Scoring_Pts_Min,Scoring_Pts_25%,Scoring_Pts_50%,Scoring_Pts_75%,Scoring_Pts_Max,History_Opp_Scoring_Pts_Count,History_Scoring_Pts_Average,History_Scoring_Pts_STD,History_Scoring_Pts_Min,History_Scoring_Pts_25%,History_Scoring_Pts_50%,History_Scoring_Pts_75%,History_Scoring_Pts_Max,Scoring_TD_Average,Scoring_TD_STD,Scoring_TD_Min,Scoring_TD_25%,Scoring_TD_50%,Scoring_TD_75%,Scoring_TD_Max,History_Opp_Scoring_TD_Count,History_Scoring_TD_Average,History_Scoring_TD_STD,History_Scoring_TD_Min,History_Scoring_TD_25%,History_Scoring_TD_50%,History_Scoring_TD_75%,History_Scoring_TD_Max,Age_Average,Age_STD,Age_Min,Age_25%,Age_50%,Age_75%,Age_Max,History_Opp_Age_Count,History_Age_Average,History_Age_STD,History_Age_Min,History_Age_25%,History_Age_50%,History_Age_75%,History_Age_Max,Non-Football Injury_Average,Non-Football Injury_STD,Non-Football Injury_Min,Non-Football Injury_25%,Non-Football Injury_50%,Non-Football Injury_75%,Non-Football Injury_Max,History_Opp_Non-Football Injury_Count,History_Non-Football Injury_Average,History_Non-Football Injury_STD,History_Non-Football Injury_Min,History_Non-Football Injury_25%,History_Non-Football Injury_50%,H

In [ ]:
sfagfes

In [ ]:
WR_Depth_Chart = pd.read_csv('WR_Depth_Chart.csv', sep = ";")

In [ ]:
WR_Depth_Chart = WR_Depth_Chart[WR_Depth_Chart["Year"] < 2023]
WR_Depth_Chart["Team"] = WR_Depth_Chart["Team"].replace("NO", "NOR").replace("LV", "LVR").replace("KC", "KAN").replace("SF", "SFO").replace("NE", "NWE").replace("GB", "GBN").replace("TB", "TAM").replace("ARZ", "ARI")
WR_Depth_Chart  = WR_Depth_Chart.reset_index(drop = True)
WR_Depth_Chart

In [ ]:
def lwr_data_org(depth):
    #name_options = filtered_data.Name_.unique()
    bad_apple = []
    players = []
    all_dfs = {}
    length = len(depth)
    count = 0
    for index, row in depth.iterrows():
        all_players = [row["1"], row["2"], row["3"], row["4"], row["5"]]
        players = [x for x in all_players if x != "0"]
        #print(players)
        #print(index, "\n")
        try:
            temp_name = players[0]
            team = row["Team"]
            month = row["Month"]
            year = row["Year"]
            #print(team, name, month, year)

            temp = filtered_data[(filtered_data["Name_"] == temp_name) & (filtered_data["Year"] == year) & (filtered_data["Month"] == month)]
            week_list = list(temp.Week)
            oppenant = list(temp.Opp)




            for index_2, week in enumerate(week_list):
                #print(players)
                #print(team, oppenant[index], players, year, week)
                try:
                    ready_data, team_data = wr_week_one(data = filtered_data,
                                                        team =team,
                                                        opp = oppenant[index_2],
                                                        players = players, 
                                                        year = year, 
                                                        week = week)

                    all_dfs[team + "_" + str(week) + "_" + str(year) + "_" + row["Pos"]] = ready_data

                except:
                    print(team, oppenant[index_2], players, year, week)
                    bad_apple.append(players)
                    #print(players)
                    count += 1
                    #print(count/length)
            if index % 1000 == 0:
                print(index/length * 100, "%")
           
        except:
            bad_apple.append(1)
                
    return all_dfs, bad_apple

In [ ]:
all_dfs, bad_apple = lwr_data_org(WR_Depth_Chart)

In [ ]:
filtered_data[(filtered_data["Name_"] == "Diggs, Stefon") & (filtered_data["Year"] == 2022) & (filtered_data["Month"] == 11)]

In [ ]:
wr_week_one(data = filtered_data,
            team =team,
            opp = oppenant[index],
            players = players, 
            year = year, 
            week = week)

In [ ]:
bad_apple